In [1]:
import re
import time
import json
import warnings
import traceback
import pandas as pd
from linebot import LineBotApi, LineBotSdkDeprecatedIn30
from linebot.v3.webhook import WebhookHandler
from linebot.models import TextSendMessage, ImageSendMessage
from datetime import datetime
from apscheduler.schedulers.background import BackgroundScheduler
from flask import Flask, request
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core._api.deprecation import LangChainDeprecationWarning
from utils.utils import Chatbot_Utils
from utils.check_user_inactivity import UserInactivity
from schema_and_template.template import langchain_system_template, langchain_condense_question_prompt, gpt_system_template, gpt_interal_tire_system_template, gpt_interal_tire_another_system_template, gpt_interal_bolt_pattern_system_template, gpt_user_data_template
from schema_and_template.schema import tools
from utils.config import channel_access_token, channel_secret

warnings.filterwarnings("ignore", category=LineBotSdkDeprecatedIn30)
warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)


scheduler = BackgroundScheduler()
scheduler.add_job(UserInactivity, 'interval', minutes=30)
scheduler.start()
app = Flask(__name__)
user_interrupt = {}
user_status = {}


@app.route("/", methods=['POST'])
def linebot():
    start_time  = time.time()
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    line_bot_api = LineBotApi(channel_access_token)
    handler = WebhookHandler(channel_secret)
    signature = request.headers['X-Line-Signature']
    handler.handle(body, signature)
    event = json_data['events'][0]
    replyToken = event['replyToken']
    user_id = event['source']['userId']
    msg_type = event['message']['type']
    
    chatbot_utils = Chatbot_Utils(user_id)
    logger = chatbot_utils.get_logger()
    logger.debug("~~啟動GPT機器人，開始記錄日誌~~")


    if '切換人工客服' in event['message']['text']:
        event['mode'] = 'standby'
        user_status[user_id] = True
        chatbot_utils.firebase_db.put('/user_status', user_id, True)
        line_bot_api.push_message(user_id, TextSendMessage(text=f'您已切換至人工客服！'))
        return 'OK', 200

    elif '切換AI客服' in event['message']['text']:
        event['mode'] = 'active'
        user_status[user_id] = False
        chatbot_utils.firebase_db.put('/user_status', user_id, False)
        line_bot_api.push_message(user_id, TextSendMessage(text='您已切換至AI客服！'))
        return 'OK', 200
    
    if user_status.get(user_id, False) or chatbot_utils.firebase_db.get('/user_status', user_id) == True:
        return 'OK', 200          

    if user_interrupt.get(user_id, False):
        line_bot_api.push_message(user_id, TextSendMessage(text=f'AI正在處理中，請等AI回覆後再輸入問題'))
        return 'OK', 200
    else:
        user_interrupt[user_id] = True

    try:
        image = None
        stored_messages = chatbot_utils.firebase_db.get('/chat', user_id)
        doc_store, llm, memory_chat = chatbot_utils.langchain_utils()
        prompt_messages = [
                            SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template=langchain_system_template)),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))
                        ]
        response = ConversationalRetrievalChain.from_llm(
                                                        llm=llm,
                                                        retriever=doc_store.as_retriever(),
                                                        return_source_documents=True,
                                                        memory=memory_chat,
                                                        combine_docs_chain_kwargs={'prompt': ChatPromptTemplate.from_messages(prompt_messages)},
                                                        condense_question_prompt=PromptTemplate.from_template(langchain_condense_question_prompt),
                                                        verbose=False
                                                        )

        if msg_type == 'text':
            user_messages = event['message']['text']
            line_bot_api.push_message(user_id, TextSendMessage(text='請稍後，我們正在處理您的請求...'))

            if stored_messages is None:
                messages = []
                messages.append({"role": "system", "content": gpt_system_template})
            else:
                messages = stored_messages

            if user_messages == '清空':
                reply_msg = TextSendMessage(text='對話歷史紀錄已經清空！')
                chatbot_utils.firebase_db.delete('/chat', user_id)
                memory_chat.clear()
                user_interrupt[user_id] = False
                logger.info("對話歷史紀錄已經清空！")

            elif re.search(r'保養|小保養|大保養|定位|基礎定位|進階定位|煞車強化', user_messages):
                logger.info("~~~~已進到langchain搜尋~~~~")
                ai_messages = response({"question": user_messages})['answer']
                messages.append({"role": "assistant", "content": ai_messages})
                reply_msg = TextSendMessage(text=ai_messages)
                chatbot_utils.firebase_db.put('/chat', user_id, messages)
                user_interrupt[user_id] = False

            elif re.search(r'查詢輪胎價格|查詢鋁圈價格', user_messages):
                logger.info("~~~~查詢價格~~~~")
                if re.search(r'查詢輪胎價格', user_messages):
                    reply_msg = TextSendMessage(text='請輸入輪胎規格')
                elif re.search(r'查詢鋁圈價格', user_messages):
                    reply_msg = TextSendMessage(text='請輸入鋁圈規格')
                user_interrupt[user_id] = False

            else:
                data = {}
                messages.append({"role": "user", "content": user_messages})
                response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages, tools=tools) 

                if response.choices[0].message.tool_calls:
                    line_bot_api.push_message(user_id, TextSendMessage(text='正在搜尋資料中，時間有可能會比較久，請稍待片刻...'))
                    logger.info(f"函數呼叫： {response}")
                    data_change = response.choices[0].message.tool_calls[0].function.name
                    logger.info(f"呼叫哪個函數： {data_change}")
                    
                    if data_change == 'fetch_tire_brand_data':
                        brand = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['brand']
                        brand_model = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['brand_model']
                        logger.info(f'未調整品牌名稱：{brand} 未調整品牌型號：{brand_model}')
                        brand_model, specification = chatbot_utils.parse_brand_model(brand_model)

                        if brand_model and specification:
                            logger.info(f'品牌名稱：{brand} 品牌型號：{brand_model} 輪胎規格：{specification}')
                            logger.info("再次確認呼叫函數：fetch_tire_all_data")
                            specification = chatbot_utils.parse_specification(specification, 'tire')
                            all_data = chatbot_utils.fetch_tire_all_data(brand, brand_model, specification)
                            tem_df = pd.DataFrame(all_data)
                            if tem_df.empty:
                                predefined_specification = chatbot_utils.parse_specification(user_messages, 'tire')
                                all_data = chatbot_utils.fetch_tire_specification_data([predefined_specification])
                                tem_df = pd.DataFrame(all_data)
                                image = chatbot_utils.upload_image(tem_df, f'./temporary_image/{predefined_specification.replace("/", "-")}')
                                urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            else:
                                image = chatbot_utils.upload_image(tem_df, f'./temporary_image/品牌名稱：{brand} 品牌型號：{brand_model} 輪胎規格：{specification.replace("/", "-")}')
                                urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            messages = chatbot_utils.format_and_prepend_messages(all_data, messages)
                            logger.info(f"Ragic data: {messages}")
                            messages.append({"role": "system", "content": gpt_interal_tire_system_template.format(urls=urls)})
                            response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                        elif specification is None:
                            logger.info(f'品牌名稱：{brand} 品牌型號：{brand_model}')
                            logger.info("再次確認呼叫函數：fetch_tire_brand_data")
                            all_data = chatbot_utils.fetch_tire_brand_data(brand, brand_model.upper())
                            tem_df = pd.DataFrame(all_data)
                            if tem_df.empty:
                                predefined_specification = chatbot_utils.parse_specification(user_messages, 'tire')
                                all_data = chatbot_utils.fetch_tire_specification_data([predefined_specification])
                                tem_df = pd.DataFrame(all_data)
                                image = chatbot_utils.upload_image(tem_df, f'./temporary_image/{predefined_specification.replace("/", "-")}')
                                urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            else:
                                image = chatbot_utils.upload_image(tem_df, f'./temporary_image/品牌名稱：{brand} 品牌型號：{brand_model}')
                                urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            messages = chatbot_utils.format_and_prepend_messages(all_data, messages)
                            logger.info(f"Ragic data: {messages}")
                            messages.append({"role": "system", "content": gpt_interal_tire_system_template.format(urls=urls)})
                            response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                        elif brand_model is None:
                            logger.info(f'品牌名稱：{brand} 品牌規格：{specification}')
                            logger.info("再次確認呼叫函數：fetch_tire_brand_specification_data")
                            all_data = chatbot_utils.fetch_tire_brand_specification_data(brand, specification)
                            tem_df = pd.DataFrame(all_data)
                            if tem_df.empty:
                                predefined_specification = chatbot_utils.parse_specification(user_messages, 'tire')
                                all_data = chatbot_utils.fetch_tire_specification_data([predefined_specification])
                                tem_df = pd.DataFrame(all_data)
                                image = chatbot_utils.upload_image(tem_df, f'./temporary_image/{predefined_specification.replace("/", "-")}')
                                urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            else:
                                image = chatbot_utils.upload_image(tem_df, f'./temporary_image/品牌名稱：{brand} 品牌規格：{specification.replace("/", "-")}')
                                urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            messages = chatbot_utils.format_and_prepend_messages(all_data, messages)
                            logger.info(f"Ragic data: {messages}")
                            messages.append({"role": "system", "content": gpt_interal_tire_system_template.format(urls=urls)})
                            response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                    elif data_change == 'fetch_tire_data':
                        make = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['make']
                        model = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['model']
                        logger.info(f"車輛資訊： 廠牌-{make} 車系-{model}")
                        all_data = chatbot_utils.fetch_tire_data(make.upper(), model.upper(), logger)
                        if all_data[0] == []:
                            logger.info("車輛資訊錯誤，改用規格搜尋資料")
                            predefined_specification = chatbot_utils.parse_specification(user_messages, 'tire')
                            all_data = chatbot_utils.fetch_tire_specification_data([predefined_specification])
                            tem_df = pd.DataFrame(all_data)
                            urls = [f"https://fourwheels.com.tw/product_tire_list.php?spec={spec}" for spec in tem_df['輪胎規格'].dropna()] if '輪胎規格' in tem_df.columns else None
                            messages = chatbot_utils.format_and_prepend_messages(all_data, messages)
                            image = chatbot_utils.upload_image(tem_df, f'./temporary_image/{predefined_specification.replace("/", "-")}')
                        else:
                            image = chatbot_utils.upload_image(pd.DataFrame(all_data[0]), f'./temporary_image/車輛資訊： 廠牌-{make} 車系-{model}')
                            urls = [f"{'https://fourwheels.com.tw/product_tire_list.php?spec='}{spec}" for spec in all_data[1]]
                            messages = chatbot_utils.format_and_prepend_messages(all_data[0], messages)
                        logger.info(f"Ragic data: {messages}")
                        messages.append({"role": "system", "content": gpt_interal_tire_system_template.format(urls=urls)})
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                    elif data_change == 'fetch_tire_single_brand_data':
                        single_brand = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['single_brand']
                        all_data = chatbot_utils.fetch_tire_single_brand_data(single_brand)
                        tem_df = pd.DataFrame(all_data)
                        urls = [f"{'https://fourwheels.com.tw/product_tire_list.php?spec='}{spec}" for spec in tem_df['輪胎規格']]
                        messages = chatbot_utils.format_and_prepend_messages(all_data, messages)
                        logger.info(f"Ragic data: {messages}")
                        messages.append({"role": "system", "content": gpt_interal_tire_system_template.format(urls=urls)})
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)
                        image = chatbot_utils.upload_image(tem_df, f'./temporary_image/({single_brand})')

                    elif data_change == 'fetch_tire_single_brand_model_data':
                        single_brand_model = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['single_brand_model']
                        all_data = chatbot_utils.fetch_tire_single_brand_model_data(single_brand_model.upper())
                        tem_df = pd.DataFrame(all_data)
                        urls = [f"{'https://fourwheels.com.tw/product_tire_list.php?spec='}{spec}" for spec in tem_df['輪胎規格']]
                        messages = chatbot_utils.format_and_prepend_messages(all_data, messages)
                        logger.info(f"Ragic data: {messages}")
                        messages.append({"role": "system", "content": gpt_interal_tire_system_template.format(urls=urls)})
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)
                        image = chatbot_utils.upload_image(tem_df, f'./temporary_image/{single_brand_model}')

                    elif data_change == 'fetch_tire_specification_data':
                        specification = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['specification']
                        specification = chatbot_utils.parse_specification(specification, 'tire')
                        tem_data = chatbot_utils.fetch_tire_specification_data([specification])
                        messages = chatbot_utils.format_and_prepend_messages(tem_data, messages)
                        logger.info(f"Ragic data: {messages}")
                        messages.append({"role": "system", "content": gpt_interal_tire_another_system_template.format(specification=specification)})
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)
                        image = chatbot_utils.upload_image(pd.DataFrame(tem_data), f'./temporary_image/{specification.replace("/", "-")}')

                    elif data_change == 'fetch_bolt_pattern_data':
                        make = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['make']
                        model = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['model']
                        year = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['year']
                        data.update({'make': make.lower(), 'model': model.lower(), 'year': str(year)})
                        logger.info(f"車輛資訊： {data}")
                        bolt_pattern_data = chatbot_utils.fetch_bolt_pattern_data(data, logger)
                        bolt_pattern_df = pd.DataFrame(bolt_pattern_data)
                        if bolt_pattern_df.empty:
                            urls = []
                        else:
                            image = chatbot_utils.upload_image(bolt_pattern_df, f'./temporary_image/車輛資訊： 廠牌-{make} 車系-{model}(鋁圈)')
                            urls = [f"https://fourwheels.com.tw/product_rim_list.php?size={row['尺寸']}&j_val={row['J數']}&hole={row['孔徑']}" for _, row in bolt_pattern_df.iterrows()] if all(column in bolt_pattern_df.columns for column in ['尺寸', 'J數', '孔徑']) else None
                        messages = chatbot_utils.format_and_prepend_messages(bolt_pattern_data, messages)
                        logger.info(f"Ragic data: {messages}")
                        messages.append({"role": "system", "content": gpt_interal_bolt_pattern_system_template.format(urls=urls)})
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                    elif data_change == 'fetch_bolt_pattern_specification_data':
                        specification = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['specification']
                        specification = chatbot_utils.parse_specification(specification, 'bolt')
                        bolt_pattern_data = chatbot_utils.fetch_bolt_pattern_specification_data(specification)
                        bolt_pattern_df = pd.DataFrame(bolt_pattern_data)
                        if bolt_pattern_df.empty:
                            urls = []
                        else:
                            image = chatbot_utils.upload_image(bolt_pattern_df, f'./temporary_image/{specification.replace("/", "-")}')
                            urls = [f"https://fourwheels.com.tw/product_rim_list.php?size={row['尺寸']}&j_val={row['J數']}&hole={row['孔徑']}" for _, row in bolt_pattern_df.iterrows()] if all(column in bolt_pattern_df.columns for column in ['尺寸', 'J數', '孔徑']) else None
                        messages = chatbot_utils.format_and_prepend_messages(bolt_pattern_data, messages)
                        logger.info(f"Ragic data: {messages}")
                        messages.append({"role": "system", "content": gpt_interal_bolt_pattern_system_template.format(urls=urls)})
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                    elif data_change == 'fetch_license_plate_number_data':
                        license_plate_number = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['license_plate_number']
                        license_plate_number = chatbot_utils.parse_license_plate_number(license_plate_number).upper()
                        logger.info(f"車牌號碼： {license_plate_number}")
                        license_plate_number_data = chatbot_utils.fetch_license_plate_number_data(license_plate_number)
                        messages.append({"role": "system", "content": gpt_user_data_template.format(
                                                                                        name=license_plate_number_data[0]['姓名'][0],
                                                                                        plate_number=license_plate_number_data[0]['車牌號碼'][0],
                                                                                        phone=license_plate_number_data[0]['電話'][0],
                                                                                        brand=license_plate_number_data[0]['車輛廠牌'][0],
                                                                                        model=license_plate_number_data[0]['車型'][0]
                                                                                    )})
                        image = chatbot_utils.upload_image(license_plate_number_data[1], f'./temporary_image/{license_plate_number}')
                        response = chatbot_utils.client.chat.completions.create(model=chatbot_utils.model_name, temperature=chatbot_utils.temperature, messages=messages)

                ai_messages = response.choices[0].message.content
                messages.append({"role": "assistant", "content": ai_messages})
                logger.info(f"完整訊息：{messages}")
                reply_msg = TextSendMessage(text=ai_messages)
                user_interrupt[user_id] = False

                now_time = datetime.now()
                compute_time = time.time() - start_time
                logger.info(f'本次回覆處理時間 {compute_time:.2f} 秒')

                chatbot_utils.firebase_db.put('/chat', user_id, messages)
                chatbot_utils.firebase_db.put('/time', user_id, now_time)

            if image:
                logger.info(f"上傳圖片的網址：{image}")
                # messages = [reply_msg, ImageSendMessage(original_content_url=image, preview_image_url=image)]
                messages = [reply_msg] + [ImageSendMessage(original_content_url=link, preview_image_url=link) for link in image]
            else:
                messages = [reply_msg]
            line_bot_api.reply_message(replyToken, messages)
        else:
            line_bot_api.reply_message(replyToken, TextSendMessage(text='請輸入文字訊息！'))

    except Exception as e:
        logger.error(e)
        traceback.print_exc()
        line_bot_api.push_message(user_id, TextSendMessage(text='資料搜尋有誤，請重新輸入正確規格！'))
        user_interrupt[user_id] = False

    return 'OK'

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8088)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8088
 * Running on http://192.168.1.125:8088
Press CTRL+C to quit
2024/07/18 12:21:07 PM - INFO : 對話歷史紀錄已經清空！
127.0.0.1 - - [18/Jul/2024 12:21:08] "POST / HTTP/1.1" 200 -
2024/07/18 12:21:12 PM - INFO : 函數呼叫： ChatCompletion(id='chatcmpl-9mCveYNJmPD3saski7KwjFbRjcnfe', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_je5wdqcJrBW0AORdxb4oLmLc', function=Function(arguments='{"specification":"225/50/17"}', name='fetch_tire_specification_data'), type='function')]))], created=1721276470, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_c4e5b6fa31', usage=CompletionUsage(completion_tokens=23, prompt_tokens=1075, total_tokens=1098))
2024/07/18 12:21:12 PM - INFO : 呼叫哪個函數： fetch_tire_specification_data
2024/07/18 12:21:25 PM - INFO : Ragic data: [{'ro

Uf26a12e3df4b5f375197ad10e6561718最近活動於2024-07-18 12:22:04.452503，未滿60分鐘，不進行操作。
